In [5]:
import pyodbc
import pandas as pd
import json

# ===========================================
# CONFIGURACIÓN DE CONEXIÓN
# ===========================================
server = "10.75.71.10,1433"
database = "UnoEE"

with open("config.json") as f:
    creds = json.load(f)

username = creds["DB_USER"]
password = creds["DB_PASS"]


df_refs = pd.read_csv("referencias.csv", dtype=str)
refs_list = df_refs["Ref"].dropna().unique().tolist()

print(f"🔎 Se cargaron {len(refs_list)} referencias del CSV")

# Construir bloque de INSERT dinámico en chunks de 1000
chunk_size = 1000
insert_stmts = []
for i in range(0, len(refs_list), chunk_size):
    chunk = refs_list[i:i+chunk_size]
    values = ", ".join([f"('{r}')" for r in chunk])
    insert_stmts.append(f"INSERT INTO #Refs(Ref) VALUES {values};")

insert_refs = "\n".join(insert_stmts)


# ===========================================
# QUERY SQL COMPLETA
# ===========================================
sql_query = f"""
SET NOCOUNT ON;

DECLARE @YearNow INT = YEAR(GETDATE());

/* ======================================================================
   0) LISTA DE REFERENCIAS (desde CSV)
   ====================================================================== */
IF OBJECT_ID('tempdb..#Refs') IS NOT NULL DROP TABLE #Refs;
CREATE TABLE #Refs(Ref VARCHAR(50) PRIMARY KEY);

{insert_refs}

DECLARE @FiltraRefs BIT = CASE WHEN EXISTS(SELECT 1 FROM #Refs) THEN 1 ELSE 0 END;

/* ======================================================================
   1) BASE DE REFERENCIAS (Principal + Alterna)
   ====================================================================== */
IF OBJECT_ID('tempdb..#BaseRefs') IS NOT NULL DROP TABLE #BaseRefs;
CREATE TABLE #BaseRefs(
  Referencia_Principal VARCHAR(50) NOT NULL PRIMARY KEY,
  Referencia_Alterna   VARCHAR(50) NULL
);

;WITH RefBase AS (
    SELECT DISTINCT t124.f124_referencia AS RefAlt, t120.f120_referencia AS RefPri
    FROM t124_mc_items_referencias t124
    JOIN t120_mc_items t120 ON t124.f124_rowid_item = t120.f120_rowid
    WHERE t124.f124_referencia <> t120.f120_referencia
    UNION 
    SELECT t120.f120_referencia, t120.f120_referencia
    FROM t120_mc_items t120
    WHERE NOT EXISTS (
        SELECT 1
        FROM t124_mc_items_referencias r
        WHERE r.f124_rowid_item = t120.f120_rowid
          AND r.f124_referencia <> t120.f120_referencia
    )
),
Filtradas AS (
    SELECT *
    FROM RefBase
    WHERE @FiltraRefs = 0
       OR RefPri IN (SELECT Ref FROM #Refs)
)
INSERT INTO #BaseRefs(Referencia_Principal, Referencia_Alterna)
SELECT
    RefPri,
    COALESCE(
        MAX(CASE WHEN RefAlt = RefPri THEN RefAlt END),
        MIN(RefAlt)
    ) AS RefAltPick
FROM Filtradas
GROUP BY RefPri;

CREATE INDEX IX_BaseRefs_Principal ON #BaseRefs(Referencia_Principal);

/* ======================================================================
   2) COMPRAS (solo importaciones aprobadas)
   ====================================================================== */
IF OBJECT_ID('tempdb..#Compras') IS NOT NULL DROP TABLE #Compras;
CREATE TABLE #Compras(
  Referencia_Principal VARCHAR(50) NOT NULL,
  Anho   INT NOT NULL,
  Origen VARCHAR(10) NOT NULL,
  Unidades  DECIMAL(18,2) NOT NULL,
  Valor_USD DECIMAL(18,2) NOT NULL
);

INSERT INTO #Compras(Referencia_Principal, Anho, Origen, Unidades, Valor_USD)
SELECT 
    i.f120_referencia AS Referencia_Principal,
    YEAR(imp.f41850_fecha) AS Anho,
    CASE 
        WHEN p.f011_id IN ('249','840') THEN 'USA'
        WHEN p.f011_id IN ('076','105') THEN 'BR'
        ELSE 'OTROS'
    END AS Origen,
    SUM(mov.f41851_cant_entrada) AS Unidades,
    SUM(mov.f41851_cant_entrada * movoc.f421_precio_unitario) AS Valor_USD
FROM t41851_import_movto          AS mov
JOIN t41850_import_docto          AS imp   ON mov.f41851_rowid_docto_import = imp.f41850_rowid
JOIN t41806_import_origen_destino AS od    ON imp.f41850_rowid_origen = od.f41806_rowid
JOIN t011_mm_paises               AS p     ON od.f41806_id_pais = p.f011_id
JOIN t121_mc_items_extensiones    AS e     ON mov.f41851_rowid_item_ext = e.f121_rowid
JOIN t120_mc_items                AS i     ON e.f121_rowid_item = i.f120_rowid
JOIN t421_cm_oc_movto             AS movoc ON mov.f41851_rowid_oc_movto = movoc.f421_rowid
JOIN #BaseRefs                    AS br    ON br.Referencia_Principal = i.f120_referencia
WHERE imp.f41850_ind_estado    = 4
  AND imp.f41850_id_tipo_docto = 'IM'
GROUP BY i.f120_referencia,
         YEAR(imp.f41850_fecha),
         CASE 
            WHEN p.f011_id IN ('249','840') THEN 'USA'
            WHEN p.f011_id IN ('076','105') THEN 'BR'
            ELSE 'OTROS'
         END;

CREATE INDEX IX_Compras ON #Compras(Referencia_Principal, Anho, Origen);

/* ======================================================================
   3) ACUMULADOS POR PERIODO
   ====================================================================== */
IF OBJECT_ID('tempdb..#AggPer') IS NOT NULL DROP TABLE #AggPer;
CREATE TABLE #AggPer(
  Referencia_Principal VARCHAR(50),
  Periodo VARCHAR(10),
  Origen  VARCHAR(10),
  U    DECIMAL(18,2),
  VUSD DECIMAL(18,2)
);

INSERT INTO #AggPer
SELECT Referencia_Principal, 'Y1', Origen,
       SUM(Unidades), SUM(Valor_USD)
FROM #Compras
WHERE Anho = @YearNow
GROUP BY Referencia_Principal, Origen;

INSERT INTO #AggPer
SELECT Referencia_Principal, 'Y2', Origen,
       SUM(Unidades), SUM(Valor_USD)
FROM #Compras
WHERE Anho BETWEEN @YearNow-1 AND @YearNow
GROUP BY Referencia_Principal, Origen;

INSERT INTO #AggPer
SELECT Referencia_Principal, 'Y3', Origen,
       SUM(Unidades), SUM(Valor_USD)
FROM #Compras
WHERE Anho BETWEEN @YearNow-2 AND @YearNow
GROUP BY Referencia_Principal, Origen;

INSERT INTO #AggPer
SELECT Referencia_Principal, 'TOTAL', Origen,
       SUM(Unidades), SUM(Valor_USD)
FROM #Compras
GROUP BY Referencia_Principal, Origen;

CREATE INDEX IX_AggPer ON #AggPer(Referencia_Principal, Periodo, Origen);

/* ======================================================================
   4) TOTALES por periodo
   ====================================================================== */
IF OBJECT_ID('tempdb..#TotPer') IS NOT NULL DROP TABLE #TotPer;
CREATE TABLE #TotPer(
  Referencia_Principal VARCHAR(50),
  Periodo VARCHAR(10),
  U_TOT    DECIMAL(18,2),
  VUSD_TOT DECIMAL(18,2)
);

INSERT INTO #TotPer
SELECT Referencia_Principal, Periodo,
       SUM(U), SUM(VUSD)
FROM #AggPer
GROUP BY Referencia_Principal, Periodo;

CREATE INDEX IX_TotPer ON #TotPer(Referencia_Principal, Periodo);

IF OBJECT_ID('tempdb..#TotWide') IS NOT NULL DROP TABLE #TotWide;
CREATE TABLE #TotWide(
  Referencia_Principal VARCHAR(50) PRIMARY KEY,
  Unidades_Y1    DECIMAL(18,2),
  Unidades_Y2    DECIMAL(18,2),
  Unidades_Y3    DECIMAL(18,2),
  Unidades_TOTAL DECIMAL(18,2),
  ValorUSD_Y1    DECIMAL(18,2),
  ValorUSD_Y2    DECIMAL(18,2),
  ValorUSD_Y3    DECIMAL(18,2),
  ValorUSD_TOTAL DECIMAL(18,2)
);

INSERT INTO #TotWide
SELECT
  t.Referencia_Principal,
  MAX(CASE WHEN t.Periodo='Y1'    THEN t.U_TOT END),
  MAX(CASE WHEN t.Periodo='Y2'    THEN t.U_TOT END),
  MAX(CASE WHEN t.Periodo='Y3'    THEN t.U_TOT END),
  MAX(CASE WHEN t.Periodo='TOTAL' THEN t.U_TOT END),
  MAX(CASE WHEN t.Periodo='Y1'    THEN t.VUSD_TOT END),
  MAX(CASE WHEN t.Periodo='Y2'    THEN t.VUSD_TOT END),
  MAX(CASE WHEN t.Periodo='Y3'    THEN t.VUSD_TOT END),
  MAX(CASE WHEN t.Periodo='TOTAL' THEN t.VUSD_TOT END)
FROM #TotPer t
GROUP BY t.Referencia_Principal;

/* ======================================================================
   5) PORCENTAJES por origen
   ====================================================================== */
IF OBJECT_ID('tempdb..#Pct') IS NOT NULL DROP TABLE #Pct;
CREATE TABLE #Pct(
  Referencia_Principal VARCHAR(50),
  Periodo VARCHAR(10),
  Origen  VARCHAR(10),
  PctUnid  DECIMAL(10,2),
  PctValor DECIMAL(10,2)
);

INSERT INTO #Pct
SELECT a.Referencia_Principal, a.Periodo, a.Origen,
       ISNULL(CAST(100.0*a.U/NULLIF(t.U_TOT,0) AS DECIMAL(10,2)),0),
       ISNULL(CAST(100.0*a.VUSD/NULLIF(t.VUSD_TOT,0) AS DECIMAL(10,2)),0)
FROM #AggPer a
JOIN #TotPer t
  ON t.Referencia_Principal = a.Referencia_Principal
 AND t.Periodo = a.Periodo;

CREATE INDEX IX_Pct ON #Pct(Referencia_Principal, Periodo, Origen);

IF OBJECT_ID('tempdb..#PctWide') IS NOT NULL DROP TABLE #PctWide;
CREATE TABLE #PctWide(
  Referencia_Principal VARCHAR(50) PRIMARY KEY,
  PctUnid_Y1_USA DECIMAL(10,2), PctUnid_Y1_BR DECIMAL(10,2), PctUnid_Y1_OTROS DECIMAL(10,2),
  PctUnid_Y2_USA DECIMAL(10,2), PctUnid_Y2_BR DECIMAL(10,2), PctUnid_Y2_OTROS DECIMAL(10,2),
  PctUnid_Y3_USA DECIMAL(10,2), PctUnid_Y3_BR DECIMAL(10,2), PctUnid_Y3_OTROS DECIMAL(10,2),
  PctUnid_TOTAL_USA DECIMAL(10,2), PctUnid_TOTAL_BR DECIMAL(10,2), PctUnid_TOTAL_OTROS DECIMAL(10,2),
  PctValor_Y1_USA DECIMAL(10,2), PctValor_Y1_BR DECIMAL(10,2), PctValor_Y1_OTROS DECIMAL(10,2),
  PctValor_Y2_USA DECIMAL(10,2), PctValor_Y2_BR DECIMAL(10,2), PctValor_Y2_OTROS DECIMAL(10,2),
  PctValor_Y3_USA DECIMAL(10,2), PctValor_Y3_BR DECIMAL(10,2), PctValor_Y3_OTROS DECIMAL(10,2),
  PctValor_TOTAL_USA DECIMAL(10,2), PctValor_TOTAL_BR DECIMAL(10,2), PctValor_TOTAL_OTROS DECIMAL(10,2)
);

INSERT INTO #PctWide
SELECT p.Referencia_Principal,
  MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctUnid END),
  MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctUnid END),
  MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctUnid END),
  MAX(CASE WHEN Periodo='Y2' AND Origen='USA'   THEN PctUnid END),
  MAX(CASE WHEN Periodo='Y2' AND Origen='BR'    THEN PctUnid END),
  MAX(CASE WHEN Periodo='Y2' AND Origen='OTROS' THEN PctUnid END),
  MAX(CASE WHEN Periodo='Y3' AND Origen='USA'   THEN PctUnid END),
  MAX(CASE WHEN Periodo='Y3' AND Origen='BR'    THEN PctUnid END),
  MAX(CASE WHEN Periodo='Y3' AND Origen='OTROS' THEN PctUnid END),
  MAX(CASE WHEN Periodo='TOTAL' AND Origen='USA'   THEN PctUnid END),
  MAX(CASE WHEN Periodo='TOTAL' AND Origen='BR'    THEN PctUnid END),
  MAX(CASE WHEN Periodo='TOTAL' AND Origen='OTROS' THEN PctUnid END),
  MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctValor END),
  MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctValor END),
  MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctValor END),
  MAX(CASE WHEN Periodo='Y2' AND Origen='USA'   THEN PctValor END),
  MAX(CASE WHEN Periodo='Y2' AND Origen='BR'    THEN PctValor END),
  MAX(CASE WHEN Periodo='Y2' AND Origen='OTROS' THEN PctValor END),
  MAX(CASE WHEN Periodo='Y3' AND Origen='USA'   THEN PctValor END),
  MAX(CASE WHEN Periodo='Y3' AND Origen='BR'    THEN PctValor END),
  MAX(CASE WHEN Periodo='Y3' AND Origen='OTROS' THEN PctValor END),
  MAX(CASE WHEN Periodo='TOTAL' AND Origen='USA'   THEN PctValor END),
  MAX(CASE WHEN Periodo='TOTAL' AND Origen='BR'    THEN PctValor END),
  MAX(CASE WHEN Periodo='TOTAL' AND Origen='OTROS' THEN PctValor END)
FROM #Pct p
GROUP BY p.Referencia_Principal;

/* ======================================================================
   6) SALIDA FINAL
   ====================================================================== */
SELECT
    br.Referencia_Principal,
    br.Referencia_Alterna,
    ISNULL(tw.Unidades_Y1,0)    AS Unidades_Y1,
    ISNULL(tw.Unidades_Y2,0)    AS Unidades_Y2,
    ISNULL(tw.Unidades_Y3,0)    AS Unidades_Y3,
    ISNULL(tw.Unidades_TOTAL,0) AS Unidades_TOTAL,
    ISNULL(tw.ValorUSD_Y1,0)    AS ValorUSD_Y1,
    ISNULL(tw.ValorUSD_Y2,0)    AS ValorUSD_Y2,
    ISNULL(tw.ValorUSD_Y3,0)    AS ValorUSD_Y3,
    ISNULL(tw.ValorUSD_TOTAL,0) AS ValorUSD_TOTAL,
    ISNULL(pw.PctUnid_Y1_USA,0)    AS PctUnid_Y1_USA,
    ISNULL(pw.PctUnid_Y1_BR,0)     AS PctUnid_Y1_BR,
    ISNULL(pw.PctUnid_Y1_OTROS,0)  AS PctUnid_Y1_OTROS,
    ISNULL(pw.PctUnid_Y2_USA,0)    AS PctUnid_Y2_USA,
    ISNULL(pw.PctUnid_Y2_BR,0)     AS PctUnid_Y2_BR,
    ISNULL(pw.PctUnid_Y2_OTROS,0)  AS PctUnid_Y2_OTROS,
    ISNULL(pw.PctUnid_Y3_USA,0)    AS PctUnid_Y3_USA,
    ISNULL(pw.PctUnid_Y3_BR,0)     AS PctUnid_Y3_BR,
    ISNULL(pw.PctUnid_Y3_OTROS,0)  AS PctUnid_Y3_OTROS,
    ISNULL(pw.PctUnid_TOTAL_USA,0) AS PctUnid_TOTAL_USA,
    ISNULL(pw.PctUnid_TOTAL_BR,0)  AS PctUnid_TOTAL_BR,
    ISNULL(pw.PctUnid_TOTAL_OTROS,0) AS PctUnid_TOTAL_OTROS,
    ISNULL(pw.PctValor_Y1_USA,0)    AS PctValor_Y1_USA,
    ISNULL(pw.PctValor_Y1_BR,0)     AS PctValor_Y1_BR,
    ISNULL(pw.PctValor_Y1_OTROS,0)  AS PctValor_Y1_OTROS,
    ISNULL(pw.PctValor_Y2_USA,0)    AS PctValor_Y2_USA,
    ISNULL(pw.PctValor_Y2_BR,0)     AS PctValor_Y2_BR,
    ISNULL(pw.PctValor_Y2_OTROS,0)  AS PctValor_Y2_OTROS,
    ISNULL(pw.PctValor_Y3_USA,0)    AS PctValor_Y3_USA,
    ISNULL(pw.PctValor_Y3_BR,0)     AS PctValor_Y3_BR,
    ISNULL(pw.PctValor_Y3_OTROS,0)  AS PctValor_Y3_OTROS,
    ISNULL(pw.PctValor_TOTAL_USA,0) AS PctValor_TOTAL_USA,
    ISNULL(pw.PctValor_TOTAL_BR,0)  AS PctValor_TOTAL_BR,
    ISNULL(pw.PctValor_TOTAL_OTROS,0) AS PctValor_TOTAL_OTROS,
    CAST(CASE WHEN ISNULL(tw.Unidades_Y1,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_Y1,
    CAST(CASE WHEN ISNULL(tw.Unidades_TOTAL,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_TOTAL
FROM #BaseRefs br
LEFT JOIN #TotWide tw ON tw.Referencia_Principal = br.Referencia_Principal
LEFT JOIN #PctWide pw ON pw.Referencia_Principal = br.Referencia_Principal
ORDER BY br.Referencia_Principal
OPTION (RECOMPILE);
"""

# ===========================================
# EJECUCIÓN Y EXPORTACIÓN
# ===========================================
try:
    connection = pyodbc.connect(
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"UID={username};"
        f"PWD={password};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=yes;"
    )
    print("✅ Conexión exitosa")

    df_result = pd.read_sql(sql_query, connection)

    output_file = "resultado_referencias.xlsx"
    df_result.to_excel(output_file, index=False)
    print(f"📂 Resultado guardado en: {output_file}")

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'connection' in locals():
        connection.close()


🔎 Se cargaron 9774 referencias del CSV
✅ Conexión exitosa


C:\Users\andres.hernandez\AppData\Local\Temp\ipykernel_27028\755397874.py:351: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(sql_query, connection)


❌ Error: 'NoneType' object is not iterable


In [7]:
import pyodbc
import pandas as pd
import json

# ===========================================
# CONFIGURACIÓN DE CONEXIÓN
# ===========================================
server = "10.75.71.10,1433"
database = "UnoEE"

with open("config.json") as f:
    creds = json.load(f)

username = creds["DB_USER"]
password = creds["DB_PASS"]

# ===========================================
# CARGA CSV CON REFERENCIAS
# ===========================================
df_refs = pd.read_csv("referencias.csv", dtype=str)
df_refs.columns = df_refs.columns.str.strip().str.lower()

refs_list = df_refs.iloc[:, 0].dropna().unique().tolist()
print(f"🔎 Se cargaron {len(refs_list)} referencias del CSV")

# Construir bloque dinámico de referencias
if refs_list:
    values = ", ".join([f"('{r}')" for r in refs_list])
    refs_cte = f"InputRefs AS (SELECT Ref FROM (VALUES {values}) v(Ref)),"
    filtra_cond = "br.Referencia_Principal IN (SELECT Ref FROM InputRefs)"
else:
    refs_cte = ""
    filtra_cond = "1=1"

# ===========================================
# QUERY SQL COMPLETA (solo lectura con CTEs)
# ===========================================
sql_query = f"""
SET NOCOUNT ON;
DECLARE @YearNow INT = YEAR(GETDATE());

WITH
{refs_cte}
RefBase AS (
    SELECT DISTINCT t124.f124_referencia AS RefAlt,
                    t120.f120_referencia AS RefPri
    FROM t124_mc_items_referencias t124
    JOIN t120_mc_items t120
      ON t124.f124_rowid_item = t120.f120_rowid
    WHERE t124.f124_referencia <> t120.f120_referencia
    UNION
    SELECT t120.f120_referencia, t120.f120_referencia
    FROM t120_mc_items t120
    WHERE NOT EXISTS (
        SELECT 1
        FROM t124_mc_items_referencias r
        WHERE r.f124_rowid_item = t120.f120_rowid
          AND r.f124_referencia <> t120.f120_referencia
    )
),
BaseRefs AS (
    SELECT RefPri AS Referencia_Principal,
           COALESCE(
               MAX(CASE WHEN RefAlt = RefPri THEN RefAlt END),
               MIN(RefAlt)
           ) AS Referencia_Alterna
    FROM RefBase
    GROUP BY RefPri
),
BaseRefsFiltradas AS (
    SELECT *
    FROM BaseRefs br
    WHERE {filtra_cond}
),
Compras AS (
    SELECT 
        i.f120_referencia AS Referencia_Principal,
        YEAR(imp.f41850_fecha) AS Anho,
        CASE 
            WHEN p.f011_id IN ('249','840') THEN 'USA'
            WHEN p.f011_id IN ('076','105') THEN 'BR'
            ELSE 'OTROS'
        END AS Origen,
        SUM(mov.f41851_cant_entrada) AS Unidades,
        SUM(mov.f41851_cant_entrada * movoc.f421_precio_unitario) AS Valor_USD
    FROM t41851_import_movto          AS mov
    JOIN t41850_import_docto          AS imp   ON mov.f41851_rowid_docto_import = imp.f41850_rowid
    JOIN t41806_import_origen_destino AS od    ON imp.f41850_rowid_origen = od.f41806_rowid
    JOIN t011_mm_paises               AS p     ON od.f41806_id_pais = p.f011_id
    JOIN t121_mc_items_extensiones    AS e     ON mov.f41851_rowid_item_ext = e.f121_rowid
    JOIN t120_mc_items                AS i     ON e.f121_rowid_item = i.f120_rowid
    JOIN t421_cm_oc_movto             AS movoc ON mov.f41851_rowid_oc_movto = movoc.f421_rowid
    JOIN BaseRefsFiltradas            AS br    ON br.Referencia_Principal = i.f120_referencia
    WHERE imp.f41850_ind_estado    = 4
      AND imp.f41850_id_tipo_docto = 'IM'
    GROUP BY i.f120_referencia,
             YEAR(imp.f41850_fecha),
             CASE 
                WHEN p.f011_id IN ('249','840') THEN 'USA'
                WHEN p.f011_id IN ('076','105') THEN 'BR'
                ELSE 'OTROS'
             END
),
AggPer AS (
    SELECT Referencia_Principal, 'Y1' AS Periodo, Origen,
           SUM(Unidades) AS U, SUM(Valor_USD) AS VUSD
    FROM Compras WHERE Anho = @YearNow
    GROUP BY Referencia_Principal, Origen
    UNION ALL
    SELECT Referencia_Principal, 'Y2', Origen,
           SUM(Unidades), SUM(Valor_USD)
    FROM Compras WHERE Anho BETWEEN @YearNow-1 AND @YearNow
    GROUP BY Referencia_Principal, Origen
    UNION ALL
    SELECT Referencia_Principal, 'Y3', Origen,
           SUM(Unidades), SUM(Valor_USD)
    FROM Compras WHERE Anho BETWEEN @YearNow-2 AND @YearNow
    GROUP BY Referencia_Principal, Origen
    UNION ALL
    SELECT Referencia_Principal, 'TOTAL', Origen,
           SUM(Unidades), SUM(Valor_USD)
    FROM Compras
    GROUP BY Referencia_Principal, Origen
),
TotPer AS (
    SELECT Referencia_Principal, Periodo,
           SUM(U) AS U_TOT, SUM(VUSD) AS VUSD_TOT
    FROM AggPer
    GROUP BY Referencia_Principal, Periodo
),
TotWide AS (
    SELECT
      t.Referencia_Principal,
      MAX(CASE WHEN t.Periodo='Y1'    THEN t.U_TOT END) AS Unidades_Y1,
      MAX(CASE WHEN t.Periodo='Y2'    THEN t.U_TOT END) AS Unidades_Y2,
      MAX(CASE WHEN t.Periodo='Y3'    THEN t.U_TOT END) AS Unidades_Y3,
      MAX(CASE WHEN t.Periodo='TOTAL' THEN t.U_TOT END) AS Unidades_TOTAL,
      MAX(CASE WHEN t.Periodo='Y1'    THEN t.VUSD_TOT END) AS ValorUSD_Y1,
      MAX(CASE WHEN t.Periodo='Y2'    THEN t.VUSD_TOT END) AS ValorUSD_Y2,
      MAX(CASE WHEN t.Periodo='Y3'    THEN t.VUSD_TOT END) AS ValorUSD_Y3,
      MAX(CASE WHEN t.Periodo='TOTAL' THEN t.VUSD_TOT END) AS ValorUSD_TOTAL
    FROM TotPer t
    GROUP BY t.Referencia_Principal
),
Pct AS (
    SELECT a.Referencia_Principal, a.Periodo, a.Origen,
           ISNULL(100.0*a.U/NULLIF(t.U_TOT,0),0) AS PctUnid,
           ISNULL(100.0*a.VUSD/NULLIF(t.VUSD_TOT,0),0) AS PctValor
    FROM AggPer a
    JOIN TotPer t
      ON t.Referencia_Principal = a.Referencia_Principal
     AND t.Periodo = a.Periodo
),
PctWide AS (
    SELECT p.Referencia_Principal,
      MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctUnid END) AS PctUnid_Y1_USA,
      MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctUnid END) AS PctUnid_Y1_BR,
      MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_Y1_OTROS,
      MAX(CASE WHEN Periodo='Y2' AND Origen='USA'   THEN PctUnid END) AS PctUnid_Y2_USA,
      MAX(CASE WHEN Periodo='Y2' AND Origen='BR'    THEN PctUnid END) AS PctUnid_Y2_BR,
      MAX(CASE WHEN Periodo='Y2' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_Y2_OTROS,
      MAX(CASE WHEN Periodo='Y3' AND Origen='USA'   THEN PctUnid END) AS PctUnid_Y3_USA,
      MAX(CASE WHEN Periodo='Y3' AND Origen='BR'    THEN PctUnid END) AS PctUnid_Y3_BR,
      MAX(CASE WHEN Periodo='Y3' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_Y3_OTROS,
      MAX(CASE WHEN Periodo='TOTAL' AND Origen='USA'   THEN PctUnid END) AS PctUnid_TOTAL_USA,
      MAX(CASE WHEN Periodo='TOTAL' AND Origen='BR'    THEN PctUnid END) AS PctUnid_TOTAL_BR,
      MAX(CASE WHEN Periodo='TOTAL' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_TOTAL_OTROS,
      MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctValor END) AS PctValor_Y1_USA,
      MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctValor END) AS PctValor_Y1_BR,
      MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctValor END) AS PctValor_Y1_OTROS,
      MAX(CASE WHEN Periodo='Y2' AND Origen='USA'   THEN PctValor END) AS PctValor_Y2_USA,
      MAX(CASE WHEN Periodo='Y2' AND Origen='BR'    THEN PctValor END) AS PctValor_Y2_BR,
      MAX(CASE WHEN Periodo='Y2' AND Origen='OTROS' THEN PctValor END) AS PctValor_Y2_OTROS,
      MAX(CASE WHEN Periodo='Y3' AND Origen='USA'   THEN PctValor END) AS PctValor_Y3_USA,
      MAX(CASE WHEN Periodo='Y3' AND Origen='BR'    THEN PctValor END) AS PctValor_Y3_BR,
      MAX(CASE WHEN Periodo='Y3' AND Origen='OTROS' THEN PctValor END) AS PctValor_Y3_OTROS,
      MAX(CASE WHEN Periodo='TOTAL' AND Origen='USA'   THEN PctValor END) AS PctValor_TOTAL_USA,
      MAX(CASE WHEN Periodo='TOTAL' AND Origen='BR'    THEN PctValor END) AS PctValor_TOTAL_BR,
      MAX(CASE WHEN Periodo='TOTAL' AND Origen='OTROS' THEN PctValor END) AS PctValor_TOTAL_OTROS
    FROM Pct p
    GROUP BY p.Referencia_Principal
)
SELECT
    br.Referencia_Principal,
    br.Referencia_Alterna,
    ISNULL(tw.Unidades_Y1,0)    AS Unidades_Y1,
    ISNULL(tw.Unidades_Y2,0)    AS Unidades_Y2,
    ISNULL(tw.Unidades_Y3,0)    AS Unidades_Y3,
    ISNULL(tw.Unidades_TOTAL,0) AS Unidades_TOTAL,
    ISNULL(tw.ValorUSD_Y1,0)    AS ValorUSD_Y1,
    ISNULL(tw.ValorUSD_Y2,0)    AS ValorUSD_Y2,
    ISNULL(tw.ValorUSD_Y3,0)    AS ValorUSD_Y3,
    ISNULL(tw.ValorUSD_TOTAL,0) AS ValorUSD_TOTAL,
    ISNULL(pw.PctUnid_Y1_USA,0)    AS PctUnid_Y1_USA,
    ISNULL(pw.PctUnid_Y1_BR,0)     AS PctUnid_Y1_BR,
    ISNULL(pw.PctUnid_Y1_OTROS,0)  AS PctUnid_Y1_OTROS,
    ISNULL(pw.PctUnid_Y2_USA,0)    AS PctUnid_Y2_USA,
    ISNULL(pw.PctUnid_Y2_BR,0)     AS PctUnid_Y2_BR,
    ISNULL(pw.PctUnid_Y2_OTROS,0)  AS PctUnid_Y2_OTROS,
    ISNULL(pw.PctUnid_Y3_USA,0)    AS PctUnid_Y3_USA,
    ISNULL(pw.PctUnid_Y3_BR,0)     AS PctUnid_Y3_BR,
    ISNULL(pw.PctUnid_Y3_OTROS,0)  AS PctUnid_Y3_OTROS,
    ISNULL(pw.PctUnid_TOTAL_USA,0) AS PctUnid_TOTAL_USA,
    ISNULL(pw.PctUnid_TOTAL_BR,0)  AS PctUnid_TOTAL_BR,
    ISNULL(pw.PctUnid_TOTAL_OTROS,0) AS PctUnid_TOTAL_OTROS,
    ISNULL(pw.PctValor_Y1_USA,0)    AS PctValor_Y1_USA,
    ISNULL(pw.PctValor_Y1_BR,0)     AS PctValor_Y1_BR,
    ISNULL(pw.PctValor_Y1_OTROS,0)  AS PctValor_Y1_OTROS,
    ISNULL(pw.PctValor_Y2_USA,0)    AS PctValor_Y2_USA,
    ISNULL(pw.PctValor_Y2_BR,0)     AS PctValor_Y2_BR,
    ISNULL(pw.PctValor_Y2_OTROS,0)  AS PctValor_Y2_OTROS,
    ISNULL(pw.PctValor_Y3_USA,0)    AS PctValor_Y3_USA,
    ISNULL(pw.PctValor_Y3_BR,0)     AS PctValor_Y3_BR,
    ISNULL(pw.PctValor_Y3_OTROS,0)  AS PctValor_Y3_OTROS,
    ISNULL(pw.PctValor_TOTAL_USA,0) AS PctValor_TOTAL_USA,
    ISNULL(pw.PctValor_TOTAL_BR,0)  AS PctValor_TOTAL_BR,
    ISNULL(pw.PctValor_TOTAL_OTROS,0) AS PctValor_TOTAL_OTROS,
    CAST(CASE WHEN ISNULL(tw.Unidades_Y1,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_Y1,
    CAST(CASE WHEN ISNULL(tw.Unidades_TOTAL,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_TOTAL
FROM BaseRefsFiltradas br
LEFT JOIN TotWide tw ON tw.Referencia_Principal = br.Referencia_Principal
LEFT JOIN PctWide pw ON pw.Referencia_Principal = br.Referencia_Principal
ORDER BY br.Referencia_Principal;
"""

# ===========================================
# EJECUCIÓN Y EXPORTACIÓN
# ===========================================
try:
    connection = pyodbc.connect(
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"UID={username};"
        f"PWD={password};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=yes;"
    )
    print("✅ Conexión exitosa")

    df_result = pd.read_sql(sql_query, connection)

    output_file = "resultado_referencias.xlsx"
    df_result.to_excel(output_file, index=False)
    print(f"📂 Resultado guardado en: {output_file}")

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'connection' in locals():
        connection.close()



🔎 Se cargaron 9774 referencias del CSV
✅ Conexión exitosa


C:\Users\andres.hernandez\AppData\Local\Temp\ipykernel_27028\4133756748.py:241: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql(sql_query, connection)


❌ Error: Execution failed on sql '
SET NOCOUNT ON;
DECLARE @YearNow INT = YEAR(GETDATE());

WITH
InputRefs AS (SELECT Ref FROM (VALUES ('47135805'), ('84146735'), ('87220475'), ('84475545'), ('87732506'), ('5183774'), ('47127081'), ('84569120'), ('84214564'), ('5194879'), ('84486692'), ('84342333'), ('20141040'), ('48055262'), ('5081170'), ('87354670'), ('5194239'), ('504380241'), ('91898881'), ('84152637'), ('84346773'), ('5802350473'), ('51410404'), ('47443632'), ('504065104'), ('84524327'), ('47367180'), ('47134874'), ('51629287'), ('5168842'), ('47134705'), ('84221215'), ('47875574'), ('84283691'), ('5802377700'), ('48138563'), ('87329736'), ('87037984'), ('84217953'), ('84257511'), ('48142231'), ('87037985'), ('47539809'), ('84592016'), ('47744116'), ('84565884'), ('47744117'), ('87220477'), ('47658867'), ('87492975'), ('87345759'), ('51626331'), ('0087254303'), ('47136923'), ('47606965'), ('47721020'), ('48132966'), ('504088268'), ('5801382396'), ('84183675'), ('84183677'), ('845

In [8]:
import pyodbc
import pandas as pd
import json

# ===========================================
# CONFIGURACIÓN DE CONEXIÓN
# ===========================================
server = "10.75.71.10,1433"
database = "UnoEE"

with open("config.json") as f:
    creds = json.load(f)

username = creds["DB_USER"]
password = creds["DB_PASS"]

# ===========================================
# CARGA CSV CON REFERENCIAS
# ===========================================
df_refs = pd.read_csv("referencias.csv", dtype=str)
refs_list = df_refs["Ref"].dropna().unique().tolist()

print(f"🔎 Se cargaron {len(refs_list)} referencias del CSV")


# ===========================================
# FUNCIÓN PARA ARMAR QUERY POR CHUNK (CTEs)
# ===========================================
def build_query(subset):
    values = ",".join([f"('{r}')" for r in subset])
    return f"""
    SET NOCOUNT ON;
    DECLARE @YearNow INT = YEAR(GETDATE());

    WITH
    InputRefs AS (
        SELECT Ref FROM (VALUES {values}) v(Ref)
    ),
    RefBase AS (
        SELECT DISTINCT t124.f124_referencia AS RefAlt, t120.f120_referencia AS RefPri
        FROM t124_mc_items_referencias t124
        JOIN t120_mc_items t120 ON t124.f124_rowid_item = t120.f120_rowid
        WHERE t124.f124_referencia <> t120.f120_referencia
        UNION 
        SELECT t120.f120_referencia, t120.f120_referencia
        FROM t120_mc_items t120
        WHERE NOT EXISTS (
            SELECT 1
            FROM t124_mc_items_referencias r
            WHERE r.f124_rowid_item = t120.f120_rowid
              AND r.f124_referencia <> t120.f120_referencia
        )
    ),
    BaseRefs AS (
        SELECT RefPri AS Referencia_Principal,
               COALESCE(MAX(CASE WHEN RefAlt = RefPri THEN RefAlt END),
                        MIN(RefAlt)) AS Referencia_Alterna
        FROM RefBase
        WHERE RefPri IN (SELECT Ref FROM InputRefs)
        GROUP BY RefPri
    ),
    Compras AS (
        SELECT 
            i.f120_referencia AS Referencia_Principal,
            YEAR(imp.f41850_fecha) AS Anho,
            CASE 
                WHEN p.f011_id IN ('249','840') THEN 'USA'
                WHEN p.f011_id IN ('076','105') THEN 'BR'
                ELSE 'OTROS'
            END AS Origen,
            SUM(mov.f41851_cant_entrada) AS Unidades,
            SUM(mov.f41851_cant_entrada * movoc.f421_precio_unitario) AS Valor_USD
        FROM t41851_import_movto          AS mov
        JOIN t41850_import_docto          AS imp   ON mov.f41851_rowid_docto_import = imp.f41850_rowid
        JOIN t41806_import_origen_destino AS od    ON imp.f41850_rowid_origen = od.f41806_rowid
        JOIN t011_mm_paises               AS p     ON od.f41806_id_pais = p.f011_id
        JOIN t121_mc_items_extensiones    AS e     ON mov.f41851_rowid_item_ext = e.f121_rowid
        JOIN t120_mc_items                AS i     ON e.f121_rowid_item = i.f120_rowid
        JOIN t421_cm_oc_movto             AS movoc ON mov.f41851_rowid_oc_movto = movoc.f421_rowid
        JOIN BaseRefs                     AS br    ON br.Referencia_Principal = i.f120_referencia
        WHERE imp.f41850_ind_estado    = 4
          AND imp.f41850_id_tipo_docto = 'IM'
        GROUP BY i.f120_referencia,
                 YEAR(imp.f41850_fecha),
                 CASE 
                    WHEN p.f011_id IN ('249','840') THEN 'USA'
                    WHEN p.f011_id IN ('076','105') THEN 'BR'
                    ELSE 'OTROS'
                 END
    ),
    AggPer AS (
        SELECT Referencia_Principal, 'Y1' AS Periodo, Origen,
               SUM(Unidades) AS U, SUM(Valor_USD) AS VUSD
        FROM Compras
        WHERE Anho = @YearNow
        GROUP BY Referencia_Principal, Origen
        UNION ALL
        SELECT Referencia_Principal, 'Y2', Origen,
               SUM(Unidades), SUM(Valor_USD)
        FROM Compras
        WHERE Anho BETWEEN @YearNow-1 AND @YearNow
        GROUP BY Referencia_Principal, Origen
        UNION ALL
        SELECT Referencia_Principal, 'Y3', Origen,
               SUM(Unidades), SUM(Valor_USD)
        FROM Compras
        WHERE Anho BETWEEN @YearNow-2 AND @YearNow
        GROUP BY Referencia_Principal, Origen
        UNION ALL
        SELECT Referencia_Principal, 'TOTAL', Origen,
               SUM(Unidades), SUM(Valor_USD)
        FROM Compras
        GROUP BY Referencia_Principal, Origen
    ),
    TotPer AS (
        SELECT Referencia_Principal, Periodo,
               SUM(U) AS U_TOT, SUM(VUSD) AS VUSD_TOT
        FROM AggPer
        GROUP BY Referencia_Principal, Periodo
    ),
    TotWide AS (
        SELECT
          t.Referencia_Principal,
          MAX(CASE WHEN t.Periodo='Y1'    THEN t.U_TOT END) AS Unidades_Y1,
          MAX(CASE WHEN t.Periodo='Y2'    THEN t.U_TOT END) AS Unidades_Y2,
          MAX(CASE WHEN t.Periodo='Y3'    THEN t.U_TOT END) AS Unidades_Y3,
          MAX(CASE WHEN t.Periodo='TOTAL' THEN t.U_TOT END) AS Unidades_TOTAL,
          MAX(CASE WHEN t.Periodo='Y1'    THEN t.VUSD_TOT END) AS ValorUSD_Y1,
          MAX(CASE WHEN t.Periodo='Y2'    THEN t.VUSD_TOT END) AS ValorUSD_Y2,
          MAX(CASE WHEN t.Periodo='Y3'    THEN t.VUSD_TOT END) AS ValorUSD_Y3,
          MAX(CASE WHEN t.Periodo='TOTAL' THEN t.VUSD_TOT END) AS ValorUSD_TOTAL
        FROM TotPer t
        GROUP BY t.Referencia_Principal
    ),
    Pct AS (
        SELECT a.Referencia_Principal, a.Periodo, a.Origen,
               ISNULL(CAST(100.0*a.U/NULLIF(t.U_TOT,0) AS DECIMAL(10,2)),0) AS PctUnid,
               ISNULL(CAST(100.0*a.VUSD/NULLIF(t.VUSD_TOT,0) AS DECIMAL(10,2)),0) AS PctValor
        FROM AggPer a
        JOIN TotPer t
          ON t.Referencia_Principal = a.Referencia_Principal
         AND t.Periodo = a.Periodo
    ),
    PctWide AS (
        SELECT p.Referencia_Principal,
          MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctUnid END) AS PctUnid_Y1_USA,
          MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctUnid END) AS PctUnid_Y1_BR,
          MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_Y1_OTROS,
          MAX(CASE WHEN Periodo='TOTAL' AND Origen='USA'   THEN PctUnid END) AS PctUnid_TOTAL_USA,
          MAX(CASE WHEN Periodo='TOTAL' AND Origen='BR'    THEN PctUnid END) AS PctUnid_TOTAL_BR,
          MAX(CASE WHEN Periodo='TOTAL' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_TOTAL_OTROS,
          MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctValor END) AS PctValor_Y1_USA,
          MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctValor END) AS PctValor_Y1_BR,
          MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctValor END) AS PctValor_Y1_OTROS,
          MAX(CASE WHEN Periodo='TOTAL' AND Origen='USA'   THEN PctValor END) AS PctValor_TOTAL_USA,
          MAX(CASE WHEN Periodo='TOTAL' AND Origen='BR'    THEN PctValor END) AS PctValor_TOTAL_BR,
          MAX(CASE WHEN Periodo='TOTAL' AND Origen='OTROS' THEN PctValor END) AS PctValor_TOTAL_OTROS
        FROM Pct p
        GROUP BY p.Referencia_Principal
    )
    SELECT
        br.Referencia_Principal,
        br.Referencia_Alterna,
        ISNULL(tw.Unidades_Y1,0)    AS Unidades_Y1,
        ISNULL(tw.Unidades_Y2,0)    AS Unidades_Y2,
        ISNULL(tw.Unidades_Y3,0)    AS Unidades_Y3,
        ISNULL(tw.Unidades_TOTAL,0) AS Unidades_TOTAL,
        ISNULL(tw.ValorUSD_Y1,0)    AS ValorUSD_Y1,
        ISNULL(tw.ValorUSD_Y2,0)    AS ValorUSD_Y2,
        ISNULL(tw.ValorUSD_Y3,0)    AS ValorUSD_Y3,
        ISNULL(tw.ValorUSD_TOTAL,0) AS ValorUSD_TOTAL,
        ISNULL(pw.PctUnid_Y1_USA,0) AS PctUnid_Y1_USA,
        ISNULL(pw.PctUnid_Y1_BR,0)  AS PctUnid_Y1_BR,
        ISNULL(pw.PctUnid_Y1_OTROS,0) AS PctUnid_Y1_OTROS,
        ISNULL(pw.PctUnid_TOTAL_USA,0) AS PctUnid_TOTAL_USA,
        ISNULL(pw.PctUnid_TOTAL_BR,0)  AS PctUnid_TOTAL_BR,
        ISNULL(pw.PctUnid_TOTAL_OTROS,0) AS PctUnid_TOTAL_OTROS,
        ISNULL(pw.PctValor_Y1_USA,0) AS PctValor_Y1_USA,
        ISNULL(pw.PctValor_Y1_BR,0)  AS PctValor_Y1_BR,
        ISNULL(pw.PctValor_Y1_OTROS,0) AS PctValor_Y1_OTROS,
        ISNULL(pw.PctValor_TOTAL_USA,0) AS PctValor_TOTAL_USA,
        ISNULL(pw.PctValor_TOTAL_BR,0)  AS PctValor_TOTAL_BR,
        ISNULL(pw.PctValor_TOTAL_OTROS,0) AS PctValor_TOTAL_OTROS,
        CAST(CASE WHEN ISNULL(tw.Unidades_Y1,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_Y1,
        CAST(CASE WHEN ISNULL(tw.Unidades_TOTAL,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_TOTAL
    FROM BaseRefs br
    LEFT JOIN TotWide tw ON tw.Referencia_Principal = br.Referencia_Principal
    LEFT JOIN PctWide pw ON pw.Referencia_Principal = br.Referencia_Principal
    ORDER BY br.Referencia_Principal;
    """


# ===========================================
# CONEXIÓN Y EJECUCIÓN POR CHUNKS
# ===========================================
try:
    connection = pyodbc.connect(
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"UID={username};"
        f"PWD={password};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=yes;"
    )
    print("✅ Conexión exitosa")

    chunk_size = 1000
    dfs = []

    for i in range(0, len(refs_list), chunk_size):
        subset = refs_list[i:i+chunk_size]
        print(f"▶ Procesando refs {i+1} a {i+len(subset)}...")
        sql_query = build_query(subset)
        df_chunk = pd.read_sql(sql_query, connection)
        dfs.append(df_chunk)

    df_result = pd.concat(dfs, ignore_index=True)

    output_file = "resultado_referencias.xlsx"
    df_result.to_excel(output_file, index=False)
    print(f"📂 Resultado final guardado en: {output_file}")

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'connection' in locals():
        connection.close()


🔎 Se cargaron 9774 referencias del CSV
✅ Conexión exitosa
▶ Procesando refs 1 a 1000...


C:\Users\andres.hernandez\AppData\Local\Temp\ipykernel_27028\543893397.py:215: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql(sql_query, connection)


▶ Procesando refs 1001 a 2000...
▶ Procesando refs 2001 a 3000...
▶ Procesando refs 3001 a 4000...
▶ Procesando refs 4001 a 5000...
▶ Procesando refs 5001 a 6000...
▶ Procesando refs 6001 a 7000...
▶ Procesando refs 7001 a 8000...
▶ Procesando refs 8001 a 9000...
▶ Procesando refs 9001 a 9774...
📂 Resultado final guardado en: resultado_referencias.xlsx


In [9]:
import pyodbc
import pandas as pd
import json

# ===========================================
# CONFIGURACIÓN DE CONEXIÓN
# ===========================================
server = "10.75.71.10,1433"
database = "UnoEE"

with open("config.json") as f:
    creds = json.load(f)

username = creds["DB_USER"]
password = creds["DB_PASS"]

# ===========================================
# CARGA CSV CON REFERENCIAS
# ===========================================
df_refs = pd.read_csv("referencias.csv", dtype=str)
refs_list = df_refs["Ref"].dropna().unique().tolist()

print(f"🔎 Se cargaron {len(refs_list)} referencias del CSV")

# ===========================================
# FUNCIÓN PARA ARMAR QUERY POR CHUNK (solo Y1 y Y3)
# ===========================================
def build_query(subset):
    values = ",".join([f"('{r}')" for r in subset])
    return f"""
    SET NOCOUNT ON;
    DECLARE @YearNow INT = YEAR(GETDATE());

    WITH
    InputRefs AS (
        SELECT Ref FROM (VALUES {values}) v(Ref)
    ),
    RefBase AS (
        SELECT DISTINCT t124.f124_referencia AS RefAlt, t120.f120_referencia AS RefPri
        FROM t124_mc_items_referencias t124
        JOIN t120_mc_items t120 ON t124.f124_rowid_item = t120.f120_rowid
        WHERE t124.f124_referencia <> t120.f120_referencia
        UNION 
        SELECT t120.f120_referencia, t120.f120_referencia
        FROM t120_mc_items t120
        WHERE NOT EXISTS (
            SELECT 1
            FROM t124_mc_items_referencias r
            WHERE r.f124_rowid_item = t120.f120_rowid
              AND r.f124_referencia <> t120.f120_referencia
        )
    ),
    BaseRefs AS (
        SELECT RefPri AS Referencia_Principal,
               COALESCE(MAX(CASE WHEN RefAlt = RefPri THEN RefAlt END),
                        MIN(RefAlt)) AS Referencia_Alterna
        FROM RefBase
        WHERE RefPri IN (SELECT Ref FROM InputRefs)
        GROUP BY RefPri
    ),
    Compras AS (
        SELECT 
            i.f120_referencia AS Referencia_Principal,
            YEAR(imp.f41850_fecha) AS Anho,
            CASE 
                WHEN p.f011_id IN ('249','840') THEN 'USA'
                WHEN p.f011_id IN ('076','105') THEN 'BR'
                ELSE 'OTROS'
            END AS Origen,
            SUM(mov.f41851_cant_entrada) AS Unidades,
            SUM(mov.f41851_cant_entrada * movoc.f421_precio_unitario) AS Valor_USD
        FROM t41851_import_movto          AS mov
        JOIN t41850_import_docto          AS imp   ON mov.f41851_rowid_docto_import = imp.f41850_rowid
        JOIN t41806_import_origen_destino AS od    ON imp.f41850_rowid_origen = od.f41806_rowid
        JOIN t011_mm_paises               AS p     ON od.f41806_id_pais = p.f011_id
        JOIN t121_mc_items_extensiones    AS e     ON mov.f41851_rowid_item_ext = e.f121_rowid
        JOIN t120_mc_items                AS i     ON e.f121_rowid_item = i.f120_rowid
        JOIN t421_cm_oc_movto             AS movoc ON mov.f41851_rowid_oc_movto = movoc.f421_rowid
        JOIN BaseRefs                     AS br    ON br.Referencia_Principal = i.f120_referencia
        WHERE imp.f41850_ind_estado    = 4
          AND imp.f41850_id_tipo_docto = 'IM'
        GROUP BY i.f120_referencia,
                 YEAR(imp.f41850_fecha),
                 CASE 
                    WHEN p.f011_id IN ('249','840') THEN 'USA'
                    WHEN p.f011_id IN ('076','105') THEN 'BR'
                    ELSE 'OTROS'
                 END
    ),
    AggPer AS (
        SELECT Referencia_Principal, 'Y1' AS Periodo, Origen,
               SUM(Unidades) AS U, SUM(Valor_USD) AS VUSD
        FROM Compras
        WHERE Anho = @YearNow
        GROUP BY Referencia_Principal, Origen
        UNION ALL
        SELECT Referencia_Principal, 'Y3', Origen,
               SUM(Unidades), SUM(Valor_USD)
        FROM Compras
        WHERE Anho BETWEEN @YearNow-2 AND @YearNow
        GROUP BY Referencia_Principal, Origen
    ),
    TotPer AS (
        SELECT Referencia_Principal, Periodo,
               SUM(U) AS U_TOT, SUM(VUSD) AS VUSD_TOT
        FROM AggPer
        GROUP BY Referencia_Principal, Periodo
    ),
    TotWide AS (
        SELECT
          t.Referencia_Principal,
          MAX(CASE WHEN t.Periodo='Y1' THEN t.U_TOT END) AS Unidades_Y1,
          MAX(CASE WHEN t.Periodo='Y3' THEN t.U_TOT END) AS Unidades_Y3,
          MAX(CASE WHEN t.Periodo='Y1' THEN t.VUSD_TOT END) AS ValorUSD_Y1,
          MAX(CASE WHEN t.Periodo='Y3' THEN t.VUSD_TOT END) AS ValorUSD_Y3
        FROM TotPer t
        GROUP BY t.Referencia_Principal
    ),
    Pct AS (
        SELECT a.Referencia_Principal, a.Periodo, a.Origen,
               ISNULL(CAST(100.0*a.U/NULLIF(t.U_TOT,0) AS DECIMAL(10,2)),0) AS PctUnid,
               ISNULL(CAST(100.0*a.VUSD/NULLIF(t.VUSD_TOT,0) AS DECIMAL(10,2)),0) AS PctValor
        FROM AggPer a
        JOIN TotPer t
          ON t.Referencia_Principal = a.Referencia_Principal
         AND t.Periodo = a.Periodo
    ),
    PctWide AS (
        SELECT p.Referencia_Principal,
          MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctUnid END) AS PctUnid_Y1_USA,
          MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctUnid END) AS PctUnid_Y1_BR,
          MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_Y1_OTROS,
          MAX(CASE WHEN Periodo='Y3' AND Origen='USA'   THEN PctUnid END) AS PctUnid_Y3_USA,
          MAX(CASE WHEN Periodo='Y3' AND Origen='BR'    THEN PctUnid END) AS PctUnid_Y3_BR,
          MAX(CASE WHEN Periodo='Y3' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_Y3_OTROS,
          MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctValor END) AS PctValor_Y1_USA,
          MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctValor END) AS PctValor_Y1_BR,
          MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctValor END) AS PctValor_Y1_OTROS,
          MAX(CASE WHEN Periodo='Y3' AND Origen='USA'   THEN PctValor END) AS PctValor_Y3_USA,
          MAX(CASE WHEN Periodo='Y3' AND Origen='BR'    THEN PctValor END) AS PctValor_Y3_BR,
          MAX(CASE WHEN Periodo='Y3' AND Origen='OTROS' THEN PctValor END) AS PctValor_Y3_OTROS
        FROM Pct p
        GROUP BY p.Referencia_Principal
    )
    SELECT
        br.Referencia_Principal,
        br.Referencia_Alterna,
        ISNULL(tw.Unidades_Y1,0) AS Unidades_Y1,
        ISNULL(tw.Unidades_Y3,0) AS Unidades_Y3,
        ISNULL(tw.ValorUSD_Y1,0) AS ValorUSD_Y1,
        ISNULL(tw.ValorUSD_Y3,0) AS ValorUSD_Y3,
        ISNULL(pw.PctUnid_Y1_USA,0) AS PctUnid_Y1_USA,
        ISNULL(pw.PctUnid_Y1_BR,0) AS PctUnid_Y1_BR,
        ISNULL(pw.PctUnid_Y1_OTROS,0) AS PctUnid_Y1_OTROS,
        ISNULL(pw.PctUnid_Y3_USA,0) AS PctUnid_Y3_USA,
        ISNULL(pw.PctUnid_Y3_BR,0) AS PctUnid_Y3_BR,
        ISNULL(pw.PctUnid_Y3_OTROS,0) AS PctUnid_Y3_OTROS,
        ISNULL(pw.PctValor_Y1_USA,0) AS PctValor_Y1_USA,
        ISNULL(pw.PctValor_Y1_BR,0) AS PctValor_Y1_BR,
        ISNULL(pw.PctValor_Y1_OTROS,0) AS PctValor_Y1_OTROS,
        ISNULL(pw.PctValor_Y3_USA,0) AS PctValor_Y3_USA,
        ISNULL(pw.PctValor_Y3_BR,0) AS PctValor_Y3_BR,
        ISNULL(pw.PctValor_Y3_OTROS,0) AS PctValor_Y3_OTROS,
        CAST(CASE WHEN ISNULL(tw.Unidades_Y1,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_Y1,
        CAST(CASE WHEN ISNULL(tw.Unidades_Y3,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_Y3
    FROM BaseRefs br
    LEFT JOIN TotWide tw ON tw.Referencia_Principal = br.Referencia_Principal
    LEFT JOIN PctWide pw ON pw.Referencia_Principal = br.Referencia_Principal
    ORDER BY br.Referencia_Principal;
    """

# ===========================================
# CONEXIÓN Y EJECUCIÓN POR CHUNKS
# ===========================================
try:
    connection = pyodbc.connect(
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"UID={username};"
        f"PWD={password};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=yes;"
    )
    print("✅ Conexión exitosa")

    chunk_size = 1000
    dfs = []

    for i in range(0, len(refs_list), chunk_size):
        subset = refs_list[i:i+chunk_size]
        print(f"▶ Procesando refs {i+1} a {i+len(subset)}...")
        sql_query = build_query(subset)
        df_chunk = pd.read_sql(sql_query, connection)
        dfs.append(df_chunk)

    df_result = pd.concat(dfs, ignore_index=True)

    output_file = "resultado_referencias_Y1_Y3.xlsx"
    df_result.to_excel(output_file, index=False)
    print(f"📂 Resultado final guardado en: {output_file}")

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'connection' in locals():
        connection.close()


🔎 Se cargaron 9774 referencias del CSV
✅ Conexión exitosa
▶ Procesando refs 1 a 1000...


C:\Users\andres.hernandez\AppData\Local\Temp\ipykernel_27028\640060626.py:194: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_chunk = pd.read_sql(sql_query, connection)


▶ Procesando refs 1001 a 2000...
▶ Procesando refs 2001 a 3000...
▶ Procesando refs 3001 a 4000...
▶ Procesando refs 4001 a 5000...
▶ Procesando refs 5001 a 6000...
▶ Procesando refs 6001 a 7000...
▶ Procesando refs 7001 a 8000...
▶ Procesando refs 8001 a 9000...
▶ Procesando refs 9001 a 9774...
📂 Resultado final guardado en: resultado_referencias_Y1_Y3.xlsx


In [2]:
import pyodbc
import pandas as pd
import json

# ===========================================
# CONFIGURACIÓN DE CONEXIÓN
# ===========================================
server = "10.75.71.10,1433"
database = "UnoEE"

with open("config.json") as f:
    creds = json.load(f)

username = creds["DB_USER"]
password = creds["DB_PASS"]

# ===========================================
# CARGA CSV CON REFERENCIAS
# ===========================================
df_refs = pd.read_csv("referencias.csv", dtype=str)
refs_list = df_refs["Ref"].dropna().unique().tolist()

print(f"🔎 Se cargaron {len(refs_list)} referencias del CSV")

# ===========================================
# FUNCIÓN PARA ARMAR QUERY POR CHUNK (solo Y1 y Y3)
# ===========================================
def build_query(subset):
    values = ",".join([f"('{r}')" for r in subset])
    return f"""
    SET NOCOUNT ON;
    DECLARE @YearNow INT = YEAR(GETDATE());

    WITH
    InputRefs AS (
        SELECT Ref FROM (VALUES {values}) v(Ref)
    ),
    RefBase AS (
        SELECT DISTINCT t124.f124_referencia AS RefAlt, t120.f120_referencia AS RefPri
        FROM t124_mc_items_referencias t124
        JOIN t120_mc_items t120 ON t124.f124_rowid_item = t120.f120_rowid
        WHERE t124.f124_referencia <> t120.f120_referencia
        UNION 
        SELECT t120.f120_referencia, t120.f120_referencia
        FROM t120_mc_items t120
        WHERE NOT EXISTS (
            SELECT 1
            FROM t124_mc_items_referencias r
            WHERE r.f124_rowid_item = t120.f120_rowid
              AND r.f124_referencia <> t120.f120_referencia
        )
    ),
    BaseRefs AS (
        SELECT RefPri AS Referencia_Principal,
               COALESCE(MAX(CASE WHEN RefAlt = RefPri THEN RefAlt END),
                        MIN(RefAlt)) AS Referencia_Alterna
        FROM RefBase
        WHERE RefPri IN (SELECT Ref FROM InputRefs)
        GROUP BY RefPri
    ),
    Compras AS (
        SELECT 
            i.f120_referencia AS Referencia_Principal,
            YEAR(imp.f41850_fecha) AS Anho,
            CASE 
                WHEN p.f011_id IN ('249','840') THEN 'USA'
                WHEN p.f011_id IN ('076','105') THEN 'BR'
                ELSE 'OTROS'
            END AS Origen,
            SUM(mov.f41851_cant_entrada) AS Unidades,
            SUM(mov.f41851_cant_entrada * movoc.f421_precio_unitario) AS Valor_USD
        FROM t41851_import_movto          AS mov
        JOIN t41850_import_docto          AS imp   ON mov.f41851_rowid_docto_import = imp.f41850_rowid
        JOIN t41806_import_origen_destino AS od    ON imp.f41850_rowid_origen = od.f41806_rowid
        JOIN t011_mm_paises               AS p     ON od.f41806_id_pais = p.f011_id
        JOIN t121_mc_items_extensiones    AS e     ON mov.f41851_rowid_item_ext = e.f121_rowid
        JOIN t120_mc_items                AS i     ON e.f121_rowid_item = i.f120_rowid
        JOIN t421_cm_oc_movto             AS movoc ON mov.f41851_rowid_oc_movto = movoc.f421_rowid
        JOIN BaseRefs                     AS br    ON br.Referencia_Principal = i.f120_referencia
        WHERE imp.f41850_ind_estado    = 4
          AND imp.f41850_id_tipo_docto = 'IM'
        GROUP BY i.f120_referencia,
                 YEAR(imp.f41850_fecha),
                 CASE 
                    WHEN p.f011_id IN ('249','840') THEN 'USA'
                    WHEN p.f011_id IN ('076','105') THEN 'BR'
                    ELSE 'OTROS'
                 END
    ),
    AggPer AS (
        SELECT Referencia_Principal, 'Y1' AS Periodo, Origen,
               SUM(Unidades) AS U, SUM(Valor_USD) AS VUSD
        FROM Compras
        WHERE Anho = @YearNow
        GROUP BY Referencia_Principal, Origen
        UNION ALL
        SELECT Referencia_Principal, 'Y3', Origen,
               SUM(Unidades), SUM(Valor_USD)
        FROM Compras
        WHERE Anho BETWEEN @YearNow-2 AND @YearNow
        GROUP BY Referencia_Principal, Origen
    ),
    TotPer AS (
        SELECT Referencia_Principal, Periodo,
               SUM(U) AS U_TOT, SUM(VUSD) AS VUSD_TOT
        FROM AggPer
        GROUP BY Referencia_Principal, Periodo
    ),
    TotWide AS (
        SELECT
          t.Referencia_Principal,
          MAX(CASE WHEN t.Periodo='Y1' THEN t.U_TOT END) AS Unidades_Y1,
          MAX(CASE WHEN t.Periodo='Y3' THEN t.U_TOT END) AS Unidades_Y3,
          MAX(CASE WHEN t.Periodo='Y1' THEN t.VUSD_TOT END) AS ValorUSD_Y1,
          MAX(CASE WHEN t.Periodo='Y3' THEN t.VUSD_TOT END) AS ValorUSD_Y3
        FROM TotPer t
        GROUP BY t.Referencia_Principal
    ),
    Pct AS (
        SELECT a.Referencia_Principal, a.Periodo, a.Origen,
               ISNULL(CAST(100.0*a.U/NULLIF(t.U_TOT,0) AS DECIMAL(10,2)),0) AS PctUnid,
               ISNULL(CAST(100.0*a.VUSD/NULLIF(t.VUSD_TOT,0) AS DECIMAL(10,2)),0) AS PctValor
        FROM AggPer a
        JOIN TotPer t
          ON t.Referencia_Principal = a.Referencia_Principal
         AND t.Periodo = a.Periodo
    ),
    PctWide AS (
        SELECT p.Referencia_Principal,
          MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctUnid END) AS PctUnid_Y1_USA,
          MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctUnid END) AS PctUnid_Y1_BR,
          MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_Y1_OTROS,
          MAX(CASE WHEN Periodo='Y3' AND Origen='USA'   THEN PctUnid END) AS PctUnid_Y3_USA,
          MAX(CASE WHEN Periodo='Y3' AND Origen='BR'    THEN PctUnid END) AS PctUnid_Y3_BR,
          MAX(CASE WHEN Periodo='Y3' AND Origen='OTROS' THEN PctUnid END) AS PctUnid_Y3_OTROS,
          MAX(CASE WHEN Periodo='Y1' AND Origen='USA'   THEN PctValor END) AS PctValor_Y1_USA,
          MAX(CASE WHEN Periodo='Y1' AND Origen='BR'    THEN PctValor END) AS PctValor_Y1_BR,
          MAX(CASE WHEN Periodo='Y1' AND Origen='OTROS' THEN PctValor END) AS PctValor_Y1_OTROS,
          MAX(CASE WHEN Periodo='Y3' AND Origen='USA'   THEN PctValor END) AS PctValor_Y3_USA,
          MAX(CASE WHEN Periodo='Y3' AND Origen='BR'    THEN PctValor END) AS PctValor_Y3_BR,
          MAX(CASE WHEN Periodo='Y3' AND Origen='OTROS' THEN PctValor END) AS PctValor_Y3_OTROS
        FROM Pct p
        GROUP BY p.Referencia_Principal
    )
    SELECT
        br.Referencia_Principal,
        br.Referencia_Alterna,
        ISNULL(tw.Unidades_Y1,0) AS Unidades_Y1,
        ISNULL(tw.Unidades_Y3,0) AS Unidades_Y3,
        ISNULL(tw.ValorUSD_Y1,0) AS ValorUSD_Y1,
        ISNULL(tw.ValorUSD_Y3,0) AS ValorUSD_Y3,
        ISNULL(pw.PctUnid_Y1_USA,0) AS PctUnid_Y1_USA,
        ISNULL(pw.PctUnid_Y1_BR,0) AS PctUnid_Y1_BR,
        ISNULL(pw.PctUnid_Y1_OTROS,0) AS PctUnid_Y1_OTROS,
        ISNULL(pw.PctUnid_Y3_USA,0) AS PctUnid_Y3_USA,
        ISNULL(pw.PctUnid_Y3_BR,0) AS PctUnid_Y3_BR,
        ISNULL(pw.PctUnid_Y3_OTROS,0) AS PctUnid_Y3_OTROS,
        ISNULL(pw.PctValor_Y1_USA,0) AS PctValor_Y1_USA,
        ISNULL(pw.PctValor_Y1_BR,0) AS PctValor_Y1_BR,
        ISNULL(pw.PctValor_Y1_OTROS,0) AS PctValor_Y1_OTROS,
        ISNULL(pw.PctValor_Y3_USA,0) AS PctValor_Y3_USA,
        ISNULL(pw.PctValor_Y3_BR,0) AS PctValor_Y3_BR,
        ISNULL(pw.PctValor_Y3_OTROS,0) AS PctValor_Y3_OTROS,
        CAST(CASE WHEN ISNULL(tw.Unidades_Y1,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_Y1,
        CAST(CASE WHEN ISNULL(tw.Unidades_Y3,0) > 0 THEN 1 ELSE 0 END AS BIT) AS TieneCompras_Y3
    FROM BaseRefs br
    LEFT JOIN TotWide tw ON tw.Referencia_Principal = br.Referencia_Principal
    LEFT JOIN PctWide pw ON pw.Referencia_Principal = br.Referencia_Principal
    ORDER BY br.Referencia_Principal;
    """

# ===========================================
# CONEXIÓN Y EJECUCIÓN POR CHUNKS
# ===========================================
try:
    connection = pyodbc.connect(
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={server};"
        f"DATABASE={database};"
        f"UID={username};"
        f"PWD={password};"
        f"Encrypt=yes;"
        f"TrustServerCertificate=yes;"
    )
    print("✅ Conexión exitosa")

    chunk_size = 1000
    dfs = []

    for i in range(0, len(refs_list), chunk_size):
        subset = refs_list[i:i+chunk_size]
        print(f"▶ Procesando refs {i+1} a {i+len(subset)}...")
        sql_query = build_query(subset)

        # Ejecutar con cursor manual (sin pd.read_sql)
        cursor = connection.cursor()
        cursor.execute(sql_query)
        rows = cursor.fetchall()
        cols = [col[0] for col in cursor.description]
        df_chunk = pd.DataFrame.from_records(rows, columns=cols)

        dfs.append(df_chunk)

    df_result = pd.concat(dfs, ignore_index=True)

    output_file = "resultado_referencias_Y1_Y3.xlsx"
    df_result.to_excel(output_file, index=False)
    print(f"📂 Resultado final guardado en: {output_file}")
    print(df_result.head())  # 👉 Opcional: ver primeras filas en consola

except Exception as e:
    print("❌ Error:", e)

finally:
    if 'connection' in locals():
        connection.close()


🔎 Se cargaron 9774 referencias del CSV
✅ Conexión exitosa
▶ Procesando refs 1 a 1000...
▶ Procesando refs 1001 a 2000...
▶ Procesando refs 2001 a 3000...
▶ Procesando refs 3001 a 4000...
▶ Procesando refs 4001 a 5000...
▶ Procesando refs 5001 a 6000...
▶ Procesando refs 6001 a 7000...
▶ Procesando refs 7001 a 8000...
▶ Procesando refs 8001 a 9000...
▶ Procesando refs 9001 a 9774...
📂 Resultado final guardado en: resultado_referencias_Y1_Y3.xlsx
                                Referencia_Principal  \
0  00100712                                      ...   
1  00100816                                      ...   
2  00100821                                      ...   
3  00100824                                      ...   
4  00101646                                      ...   

                                  Referencia_Alterna Unidades_Y1 Unidades_Y3  \
0  00100712                                      ...      2.0000      2.0000   
1  00100816                                      ...  